# Truncate All Tables in Selected Schemas
This notebook truncates (removes all rows from) tables in the schema `sales` if they exist.

In [ ]:
# Use widgets for catalog and schema selection (Databricks UI/job configurable)
dbutils.widgets.text('catalog_name', 'maag_adb2')
dbutils.widgets.text('schema_name', 'sales')
CATALOG_NAME = dbutils.widgets.get('catalog_name')
SCHEMA_NAME = dbutils.widgets.get('schema_name')


# Truncate tables in the selected schema if they exist (Databricks version)
def truncate_tables(catalog_name, schema_name, tables):
    try:
        existing_tables = set(spark.sql(f"SHOW TABLES IN {catalog_name}.{schema_name}").select('tableName').rdd.flatMap(lambda x: x).collect())
    except Exception as e:
        print(f"⚠️ Schema {catalog_name}.{schema_name} not found. Skipping all tables in this schema.")
        return
    for table in tables:
        full_table = f"{catalog_name}.{schema_name}.{table}"
        if table in existing_tables:
            print(f"🔨 Truncating {full_table} ...")
            try:
                spark.sql(f"TRUNCATE TABLE {full_table}")
                print(f"✅ {full_table} truncated!")
            except Exception as e:
                print(f"⚠️ Could not truncate {full_table}: {e}")
        else:
            print(f"⚠️ Table {full_table} does not exist. Skipping.")


# Define tables to truncate in the selected schema
tables = ["order", "orderline", "orderpayment"]


# Truncate all tables in the selected schema
print(f"\n--- Truncating tables in schema: {CATALOG_NAME}.{SCHEMA_NAME} ---")
truncate_tables(CATALOG_NAME, SCHEMA_NAME, tables)


print(f"\n🎉 ALL SELECTED TABLES TRUNCATED in {CATALOG_NAME}.{SCHEMA_NAME}!")